# Regression to Predict the runs scored given the type of bowlers expected to bowl and the state of the match

Here, we will consider all the data available as we trying to get a relation between the strike rate, and deliveries per wicket vs each type of bowler vs the expeted runs scored 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('../../Data/selected_data/processed_data.csv')

# merging medium bowlers to fast
data.loc[data['bowler_type'] == 'Left arm Medium', 'bowler_type'] = 'Left arm Fast'
data.loc[data['bowler_type'] == 'Right arm Medium', 'bowler_type'] = 'Right arm Fast'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217242 entries, 0 to 217241
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   batter               217242 non-null  object 
 1   bowler               217242 non-null  object 
 2   non_striker          217242 non-null  object 
 3   runs_by_bat          217242 non-null  int64  
 4   extra_runs           217242 non-null  int64  
 5   total_runs_delivery  217242 non-null  int64  
 6   current_team_total   217242 non-null  int64  
 7   runs_remain          217242 non-null  float64
 8   batter_runs          217242 non-null  int64  
 9   balls_faced          217242 non-null  int64  
 10  wickets_fallen       217242 non-null  int64  
 11  extra_type           217242 non-null  object 
 12  delivery             217242 non-null  float64
 13  over                 217242 non-null  int64  
 14  wicket_type          217242 non-null  object 
 15  player_out       

In [4]:
def getPlayerScores(player_name: str, innings: list[int] = [1, 2] ) -> pd.DataFrame:
    # Get the data for BKG Mendis if batter is BKG Mendis or non-striker is BKG Mendis
	player_data = data.loc[
		((data['batter'] == player_name) | (data['non_striker'] == player_name)) & (data['innings'].isin(innings))
	]

	player_data.head()

	# 3 matches missing from the data
	# group data by match_id
	gp = player_data.groupby('match_id')
	cols = [ 'batter', 'non_striker', 'batter_runs', 'balls_faced', 'wicket_type', 'won', 'innings', 'over', 'delivery', 'wickets_fallen', 'bowling_team']
	player_scores= gp.last().loc[:, cols]

	# get the first ball he faced or at non-striker
	first_ball = gp.first().loc[:, ['over', 'delivery', 'wickets_fallen']]
	first_ball['first_ball'] = (first_ball['over'] * 6 + first_ball['delivery']).astype(int)

	player_scores['first_ball'] = first_ball['first_ball']
	player_scores['wickets_fallen'] = first_ball['wickets_fallen']

	# when BKG Mendis is the non-striker when the last ball was bowled
	# The batter_runs and balls_faced are not his, but the on_strike batter's
	# So, we need to get the last ball he faced
	# he might not even have faced a ball

	# get the last ball he faced

	matches_non_striker = player_scores[player_scores['non_striker'] == player_name].index
	
	# Sometimes the player might not even have faced a single ball
	# Eg: Afghanistan_Sri Lanka_2022-11-01 MD Shanaka not out on the non strikers end

	player_scores.loc[matches_non_striker, ['batter_runs', 'balls_faced']] = [0, 0]
	
	
	# get the last batter == player_name row from gp data
	gp = player_data[(player_data['batter'] == player_name) & (player_data['match_id'].isin(matches_non_striker))].groupby(['match_id'])
	last_batter_scores = gp.last()[['batter_runs', 'balls_faced']]	
	
	# update the rows with non_striker with correct values
	player_scores.update(last_batter_scores)
	
	



	# adding new features
	# strike rate
	player_scores['strike_rate'] = round(player_scores['batter_runs'] / player_scores['balls_faced'] * 100, 2)
	player_scores['out'] = player_scores['wicket_type'] != '0'
	player_scores['last_ball'] = (player_scores['over'] * 6 + player_scores['delivery']).astype(int)

	# drop over and delivery
	player_scores.drop(['over', 'delivery'], inplace=True, axis=1)


	# concatenating the remaining bowler types number to the dataset
	matches = data[data['match_id'].isin(player_scores.index)]
	# matches = matches[matches['batting_team'] == 'Sri Lanka']
	cols = ['match_id', 'batter', 'non_striker', 'bowler_type', 'batter_runs', 'balls_faced', 'wicket_type', 'won', 'innings', 'over', 'delivery', 'wickets_fallen']
	matches = matches[cols]
	matches['ball_number'] = (matches['over'] * 6 + matches['delivery']).astype(int)
	matches.drop(['over', 'delivery'], inplace=True, axis = 1)


	def filter_by_player_and_ball_number(group):
		player_data = group[group['batter'] == player_name]
		
		if player_data.empty:
			return player_data.drop('match_id', axis=1)
		
		first_ball_number = player_data['ball_number'].iloc[0]

		# return player_data[player_data['ball_number'] > first_ball_number].drop('match_id', axis=1) # This is for number of balls faced
		
		# fixed error should be greater or equal
		remaining = group[group['ball_number'] >= first_ball_number].drop('match_id', axis=1) # return the remianing number of deliveries for each type
		return remaining

	gp = matches.groupby('match_id').apply(filter_by_player_and_ball_number)
	remaining_ball_types = gp.groupby('match_id')['bowler_type'].value_counts().unstack(fill_value = 0)
	remaining_ball_types = remaining_ball_types.reset_index()

	player_scores = player_scores.merge(remaining_ball_types, how='left', on='match_id')

	### Attaching the player_v_bowler results to this dataset
	player_vs_bowler = get_player_v_bowlers(player_name)
	


	bowler_types = ['Left arm Fast', 'Right arm Fast', 'Left arm Orthodox', 'Left arm Wrist spin', 'Right arm Legbreak', 'Right arm Offbreak']
	for bowler_type in bowler_types:
			if bowler_type in player_vs_bowler.index:
				player_scores[f'{bowler_type} Expected Runs'] = player_vs_bowler.loc[bowler_type, 'strike_rate'] / 100 *  player_scores[bowler_type]
				player_scores[f'{bowler_type} Expected Wickets'] = player_scores[bowler_type] / player_vs_bowler.loc[bowler_type, 'deliveries_per_wicket']
			else:
				player_scores[bowler_type] = 0
				player_scores[f'{bowler_type} Expected Runs'] = 0
				player_scores[f'{bowler_type} Expected Wickets'] = 0
	
	# player_scores.fillna(0, inplace=True)
	return player_scores

def get_player_v_bowlers(player_name: str, innings = [1, 2]) -> pd.DataFrame:
	player_data = data.loc[
			(data['batter'] == player_name) & (data['innings'].isin(innings))
		]

	player_data.head()

	# Convert all medium bowlers to fast
	player_data.loc[player_data['bowler_type'] == 'Left arm Medium', 'bowler_type'] = 'Left arm Fast'
	player_data.loc[player_data['bowler_type'] == 'Right arm Medium', 'bowler_type'] = 'Right arm Fast'


	player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out') 


	cols = ['batter', 'non_striker', 'runs_by_bat', 'out', 'won', 'innings', 'over', 'delivery','bowler_type']
	player_data = player_data[cols]

	gp = player_data.groupby('bowler_type')

	player_v_bowler = pd.DataFrame()
	player_v_bowler['strike_rate'] = round(gp['runs_by_bat'].mean() * 100, 3)
	player_v_bowler['strike_rate_std'] = gp['runs_by_bat'].std()
	player_v_bowler['wickets'] = gp['out'].sum()
	player_v_bowler['deliveries_per_wicket'] = round(1 / gp['out'].mean(), 3)
	player_v_bowler['deliveries'] = gp.size()
	return player_v_bowler



In [7]:
selected_batters = ["PWH de Silva",'KIC Asalanka','BKG Mendis',"P Nissanka",'PHKD Mendis','S Samarawickrama','AD Mathews','MD Shanaka','DM de Silva','M Theekshana','PVD Chameera','N Thushara','M Pathirana','D Madushanka']

merged_df = pd.DataFrame()

for player in data['batter'].unique():
    print("Analyzing Player", player)
    
    player_scores = getPlayerScores(player)
    player_scores['batter'] = player
    player_scores.drop('non_striker', inplace=True, axis = 1)
    merged_df = pd.concat([merged_df, player_scores])



Analyzing Player AJ Finch


C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


Analyzing Player M Klinger
Analyzing Player TM Head


C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


Analyzing Player MC Henriques
Analyzing Player AJ Turner


...

C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWith

Analyzing Player TB de Bruyn
Analyzing Player ST de Silva
Analyzing Player Shuvagata Hom


C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWith

Analyzing Player Muktar Ali
Analyzing Player Khurram Manzoor


C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_34096\3623498637.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


In [6]:
merged_df.info()

NameError: name 'merged_df' is not defined

In [115]:

targets = ['batter_runs', 'balls_faced', 'last_ball', 'strike_rate']
target_df = merged_df[targets]

input_df = merged_df.drop(targets, axis=1)
input_df.drop(['wicket_type', 'out', 'match_id', 'won'], inplace=True, axis = 1)

# Preprocess the data
input_df = pd.get_dummies(data=input_df,dtype=int)


In [105]:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())

In [116]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

batter_runs = target_df['balls_faced']
tree_reg = DecisionTreeRegressor()

scores = cross_val_score(tree_reg, input_df, batter_runs, scoring="neg_mean_squared_error", cv = 10)
tree_rmse_scores = np.sqrt(-scores)



display_scores(tree_rmse_scores)
    


Scores:  [15.29592235 16.2660246  16.95440567 16.6766993  14.3989193  14.39578537
 14.60049538 13.73202687 15.01958588 12.55457651]
Mean:  14.989444121298604
Standard Deviation:  1.2959362990598806


In [107]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
scores = cross_val_score(forest_reg, input_df, batter_runs, scoring="neg_mean_squared_error", cv = 10)
tree_rmse_scores = np.sqrt(-scores)



display_scores(tree_rmse_scores)

Scores:  [11.16750809 11.62762142 11.90536898 12.34034615 10.36359264 10.49250452
 10.6286372   9.62422484 10.98268734  8.90914045]
Mean:  10.804163162433658
Standard Deviation:  0.9858952667260973


In [127]:
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.ensemble import RandomForestRegressor # type: ignore
from sklearn.metrics import mean_squared_error# type: ignore
RANDOM_STATE = 42

input_df.fillna(value=0, inplace=True)
batter_runs.fillna(value=0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(input_df, batter_runs,test_size=0.2, random_state=RANDOM_STATE)

forest_reg = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
forest_reg.fit(X_train, y_train)

y_pred = forest_reg.predict(X_test)
forest_mse = mean_squared_error(y_test, y_pred)
forest_rmse = np.sqrt(forest_mse)
print("RMSE: ", forest_rmse)


C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\736602001.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_runs.fillna(value=0, inplace=True)


10.920949818038016

In [130]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(input_df, batter_runs,test_size=0.2, random_state=RANDOM_STATE)

regr = make_pipeline(StandardScaler(), SVR(C= 1.0, epsilon=0.2))
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

svr_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
svr_rmse

11.693987246643676

GradientBoosting Model

In [148]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(input_df, batter_runs, test_size=0.2, random_state=42)

model = GradientBoostingRegressor(n_estimators=100, )

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate model performance
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Root Mean Squared Error (RMSE): 10.74


In [218]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(input_df, batter_runs, test_size=0.2, random_state=42)

model = XGBRegressor(learning_rate=0.1, n_estimators=100, max_depth=5,
                     min_child_weight=1, gamma=0, reg_lambda=2, alpha=60,
                     colsample_bytree=1, subsample=1)
# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate model performance
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Root Mean Squared Error (RMSE): 10.75


Considering only Sri Lankan batsmen

In [131]:
selected_batters = ["PWH de Silva",'KIC Asalanka','BKG Mendis',"P Nissanka",'PHKD Mendis','S Samarawickrama','AD Mathews','MD Shanaka','DM de Silva','M Theekshana','PVD Chameera','N Thushara','M Pathirana','D Madushanka']

sl_df = pd.DataFrame()

for player in selected_batters:
    print("Analyzing Player", player)
    
    player_scores = getPlayerScores(player)
    player_scores['batter'] = player
    player_scores.drop('non_striker', inplace=True, axis = 1)
    sl_df = pd.concat([merged_df, player_scores])
    

targets = ['batter_runs', 'balls_faced', 'last_ball', 'strike_rate']
sl_targets = merged_df[targets]

sl_df = merged_df.drop(targets, axis=1)
sl_df.drop(['wicket_type', 'out', 'match_id', 'won'], inplace=True, axis = 1)

# Preprocess the data
sl_df = pd.get_dummies(data=sl_df,dtype=int)
sl_batter_runs = sl_targets['batter_runs']


X_train, X_test, y_train, y_test = train_test_split(sl_df, sl_batter_runs,test_size=0.2, random_state=RANDOM_STATE)

forest_reg = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
forest_reg.fit(X_train, y_train)

y_pred = forest_reg.predict(X_test)
forest_mse = mean_squared_error(y_test, y_pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

Analyzing Player PWH de Silva


C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


Analyzing Player KIC Asalanka
Analyzing Player BKG Mendis


C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


Analyzing Player P Nissanka
Analyzing Player PHKD Mendis


...

C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


Analyzing Player PVD Chameera
Analyzing Player N Thushara


C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCop

Analyzing Player M Pathirana
Analyzing Player D Madushanka


18.03434259084105

Training model per player

In [138]:
single_player_df = pd.DataFrame()

player = 'BKG Mendis'
player_scores = getPlayerScores(player)
player_scores['batter'] = player
player_scores.drop('non_striker', inplace=True, axis = 1)
single_player_df = pd.concat([single_player_df, player_scores])



targets = ['batter_runs', 'balls_faced', 'last_ball', 'strike_rate']
target_df = single_player_df[targets]

single_player_df = single_player_df.drop(targets, axis=1)
single_player_df.drop(['wicket_type', 'out', 'match_id', 'won'], inplace=True, axis = 1)

# Preprocess the data
single_player_df = pd.get_dummies(data=single_player_df,dtype=int)

forest_reg = RandomForestRegressor(n_estimators=1000, random_state=RANDOM_STATE)
scores = cross_val_score(forest_reg, single_player_df, target_df['batter_runs'], scoring="neg_mean_squared_error", cv = 10)
tree_rmse_scores = np.sqrt(-scores)



display_scores(tree_rmse_scores)

C:\Users\kaush\AppData\Local\Temp\ipykernel_29932\521095522.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')


Scores:  [18.18379011 20.83205842 21.12163118 21.29701832 22.15993948  9.85066378
 21.51811969 22.06749577 40.05011391 20.6088058 ]
Mean:  21.768963647032802
Standard Deviation:  7.0161971992616
